In [2]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn import functional as F
import re

%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [3]:
vanilla_SC = [82.83, 82.74]
vanilla_C = [73.13]
vanilla_C_SC = [82.82]
MultiplexNet_C = [74.78, 74.17, 75.29, 74.5]
MultiplexNet_SC = [84.19, 83.61, 84.32, 83.78]

In [16]:
round(np.mean(vanilla_C),1), round(np.std(vanilla_C),1)

(73.1, 0.0)

In [17]:
round(np.mean(vanilla_C_SC),1), round(np.std(vanilla_C_SC),1)

(82.8, 0.0)

In [11]:
round(np.mean(vanilla_SC),1), round(np.std(vanilla_SC), 1)

(82.8, 0.0)

In [14]:
round(np.mean(MultiplexNet_C),1), round(np.std(MultiplexNet_C),1)

(74.7, 0.4)

In [15]:
round(np.mean(MultiplexNet_SC),1), round(np.std(MultiplexNet_SC),1)

(84.0, 0.3)